In [0]:
%fs ls /mnt/dsa/raw/dsa/daily_sales_aggregate/v2/load_date=2024-06-11/2024-06-11/

In [0]:
%fs ls /mnt/gold/release/daily_sales/2024_06_11/csv/

In [0]:
%fs ls /mnt/gold/release_blue/sales/2024_06_11/csv/

In [0]:
dfo= spark.read.csv("/mnt/gold/release/daily_sales/2024_06_06/csv/101_Daily_Sales.540.20240606061502.csv",header=True)
dfo.createOrReplaceTempView("oldfile")

In [0]:
dfn= spark.read.csv("/mnt/gold/release_blue/sales/2024_06_06/csv/101_Daily_Sales.540.20240606061502.csv",header=True)
dfn.createOrReplaceTempView("newfile")


In [0]:
%sql select 'old', count(1) from oldfile
union all select 'new', count(1) from newfile

In [0]:
%sql select * from oldfile
minus select * from newfile

In [0]:
%sql select  DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE,cast(POS_NET_DOL_AMOUNT as decimal(8,3)),cast(POS_UOM_QTY as decimal(18,3)), POS_UNITS from newfile 
minus select   DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE,cast(POS_NET_DOL_AMOUNT as decimal(8,3)),cast(POS_UOM_QTY as decimal(18,3)), POS_UNITS from oldfile  

In [0]:
%sql select DIV,STORE,ITEM_CONSUMER_UPC,sum(cast(POS_NET_DOL_AMOUNT as decimal(8,2))),sum(cast(POS_UOM_QTY as decimal(18,3))), sum(POS_UNITS) from oldfile group by DIV,STORE,ITEM_CONSUMER_UPC
minus select DIV,STORE,ITEM_CONSUMER_UPC,sum(cast(POS_NET_DOL_AMOUNT as decimal(8,2))),sum(cast(POS_UOM_QTY as decimal(18,3))), sum(POS_UNITS) from newfile group by DIV,STORE,ITEM_CONSUMER_UPC

In [0]:
%sql select 'new', DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE,sum(cast(POS_NET_DOL_AMOUNT as decimal(8,3))),sum(cast(POS_UOM_QTY as decimal(18,3))), sum(POS_UNITS) from newfile group by DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE
union all select  'old', DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE,sum(cast(POS_NET_DOL_AMOUNT as decimal(8,3))),sum(cast(POS_UOM_QTY as decimal(18,3))), sum(POS_UNITS) from oldfile  group by DIV,STORE,ITEM_CONSUMER_UPC,POS_DATE
order by store